In [2]:
# part 1

from qiskit_ibm_runtime import QiskitRuntimeService

service=QiskitRuntimeService()

backend=service.backend("ibm_brisbane")
path_answer=list(range(10))

In [17]:
from qc_grader.challenges.qgss_2024 import *

from math import pi
from qiskit.circuit.library import QFT
from qiskit import QuantumCircuit
from qiskit.providers.fake_provider import GenericBackendV2, generic_backend_v2
generic_backend_v2._NOISE_DEFAULTS["cx"] = (5.99988e-06, 6.99988e-06, 1e-5, 5e-3)

from qiskit import transpile, QuantumCircuit
from qiskit.circuit import Gate
from qiskit.converters import circuit_to_dag
from qiskit.transpiler import CouplingMap, StagedPassManager, PassManager, AnalysisPass, TransformationPass
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler.preset_passmanagers.common import generate_unroll_3q, generate_embed_passmanager
from qiskit.quantum_info import hellinger_fidelity
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.dagcircuit import DAGCircuit
from qiskit_ibm_runtime.fake_provider import FakeTorino

In [3]:
from qc_grader.challenges.qgss_2024 import grade_lab2_ex1
grade_lab2_ex1(path_answer,backend)

Submitting your answer. Please wait...
Nice work, all your answers look correct!


In [12]:
layer_1=[]
layer_2=[]

for counter in range (len(path_answer)-1):
    pair=[counter,counter+1]
    if tuple(pair) not in list(backend.coupling_map):
        pair=tuple(reversed(pair))
    if counter%2==0:
        layer_1.append(tuple(pair))
    else:
        layer_2.append(tuple(pair))

print(layer_1)

[(1, 0), (3, 2), (4, 5), (6, 7), (8, 9)]


In [13]:
from qc_grader.challenges.qgss_2024 import grade_lab2_ex2
grade_lab2_ex2(layer_1, layer_2, path_answer, backend)

Submitting your answer. Please wait...
Nice work, all your answers look correct!


In [16]:
backend.operation_names

['id',
 'rz',
 'sx',
 'x',
 'ecr',
 'reset',
 'measure',
 'switch_case',
 'for_loop',
 'if_else',
 'delay']

In [18]:
gate_name="ecr"

circuit_1=QuantumCircuit(backend.num_qubits)
circuit_2=QuantumCircuit(backend.num_qubits)

for circuit, layer in [(circuit_1,layer_1),(circuit_2,layer_2)]:
    for (i,j) in layer:
        circuit.ecr(i,j)

In [19]:
grade_lab2_ex3(circuit_1,layer_1,gate_name,backend)
grade_lab2_ex3(circuit_2,layer_2,gate_name,backend)

Submitting your answer. Please wait...
Nice work, all your answers look correct!
Submitting your answer. Please wait...
Nice work, all your answers look correct!


In [20]:
from qiskit.circuit import Parameter
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

def eplg_circuit(num_qubits: int, depth: int, layer: QuantumCircuit, qubits: tuple[int, ...]) -> QuantumCircuit:
    if depth % 2 != 0:
        raise ValueError(f"The depth must be even, got {depth}")

    def parameters():
        _depth = 0
        while True:
            for zi in range(3):
                for q in range(num_qubits):
                    yield Parameter(f"d{_depth}_q{q}_z{zi}")
            _depth += 1
    _parameters = parameters()

    circ = QuantumCircuit(num_qubits)

    def _1q_layer():
        for j in range(3):
            for k in qubits:
                circ.rz(next(_parameters), k)
            if j == 2:
                continue
            circ.sx(qubits)

    for _depth in range(depth // 2):
        circ.barrier(qubits)
        _1q_layer()
        circ.barrier(qubits)
        circ.compose(layer, inplace=True)
    circ.barrier(qubits)
    _1q_layer()
    circ.barrier(qubits)
    
    # Assuming mirroring is fine
    circ = circ.compose(circ.inverse())

    circ.measure_active()

    pm = generate_preset_pass_manager(basis_gates=["ecr", "sx", "rz"], optimization_level=0)
    circ = pm.run(circ)

    circ.metadata["layer_depth"] = depth
    circ.metadata["qubits"] = qubits
    circ.metadata["num_qubits"] = num_qubits

    return circ

In [25]:
example_circuit_1 = eplg_circuit(
    num_qubits=backend.num_qubits,
    depth=4,
    layer=circuit_1,
    qubits=path_answer,
)


In [21]:
import numpy as np

from qiskit.primitives.containers.bindings_array import BindingsArray

def get_clifford_rz_samples(
    circ: QuantumCircuit,
    num_samples: int,
    rng: np.random.Generator,
) -> BindingsArray:
    """Uniformly sample the Rz parameters in a `QuantumCircuit`
    from -pi, -pi/2, 0, +pi/2, +pi.
    """

    _allowed_ops = {'rz', 'sx', 'sxdg', 'ecr', 'barrier', 'measure'}
    if (_other := set(circ.count_ops().keys()) - _allowed_ops):
        raise ValueError(f"Circuit must only contain ops: {_allowed_ops}, got {_other}")

    sampled_pars = rng.integers(-2, 3, size=(num_samples, len(circ.parameters))) * (np.pi/2)

    return BindingsArray({
        tuple(circ.parameters): sampled_pars
    })

In [26]:
num_samples = 10
ARRAY_1_ANSWER = get_clifford_rz_samples(
    circ=example_circuit_1,
    num_samples=num_samples,
    rng=np.random.default_rng(42))

In [27]:
grade_lab2_ex4(ARRAY_1_ANSWER,example_circuit_1,num_samples)

Submitting your answer. Please wait...
Nice work, all your answers look correct! 
 Notice that the shape of the BindingsArray is [10], and there are {num_parameters} parameters.
